In [ ]:
"""
Purpose: Figure out the following: 
1) Had to add a second point to the output
2) Why in high crossovers some nodes that will be errored out are put in blue

"""

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import datajoint as dj
import trimesh
from tqdm.notebook import tqdm
from pathlib import Path

from os import sys
sys.path.append("/meshAfterParty/")
sys.path.append("/meshAfterParty/meshAfterParty")

import datajoint_utils as du
from importlib import reload

INFO - 2021-05-12 16:04:46,677 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-05-12 16:04:46,679 - settings - Setting database.user to celiib
INFO - 2021-05-12 16:04:46,680 - settings - Setting database.password to newceliipass
INFO - 2021-05-12 16:04:46,684 - settings - Setting stores to {'minnie65': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65', 'stage': '/mnt/dj-stor01/platinum/minnie65'}, 'meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/meshes'}, 'decimated_meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes'}, 'skeletons': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/skeletons'}}
INFO - 2021-05-12 16:04:46,685 - settings - Setting enable_python_native_blobs to True
INFO - 2021-05-12 16:04:46,696 - connection - Connect

Connecting celiib@at-database.ad.bcm.edu:3306


INFO - 2021-05-12 16:04:47,180 - settings - Setting enable_python_native_blobs to True
INFO - 2021-05-12 16:04:47,183 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-05-12 16:04:47,184 - settings - Setting database.user to celiib
INFO - 2021-05-12 16:04:47,185 - settings - Setting database.password to newceliipass
INFO - 2021-05-12 16:04:47,186 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-05-12 16:04:47,187 - settings - Setting database.user to celiib
INFO - 2021-05-12 16:04:47,187 - settings - Setting database.password to newceliipass
INFO - 2021-05-12 16:04:47,190 - settings - Setting enable_python_native_blobs to True
INFO - 2021-05-12 16:04:47,684 - settings - Setting enable_python_native_blobs to True


In [3]:
import neuron_visualizations as nviz
import neuron_utils as nru
import proofreading_utils as pru
import time
import system_utils as su
import axon_utils as au
import trimesh_utils as tu
import allen_utils as alu

In [4]:
minnie,_ = du.configure_minnie_vm()

INFO - 2021-05-12 16:04:48,310 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-05-12 16:04:48,311 - settings - Setting database.user to celiib
INFO - 2021-05-12 16:04:48,312 - settings - Setting database.password to newceliipass
INFO - 2021-05-12 16:04:48,315 - settings - Setting enable_python_native_blobs to True
INFO - 2021-05-12 16:04:48,879 - settings - Setting enable_python_native_blobs to True


# Getting the Neuron

In [5]:
segment_id = 864691134884759034

In [6]:
neuron_obj = du.decomposition_with_spine_recalculation(segment_id,split_index=0)

**Using table __decomposition_axon for table_to_neuron_objs**
Decompressing Neuron in minimal output mode...please wait
Dont need spine recalculation
Number of Neurons found = 1


In [7]:
nviz.visualize_neuron(neuron_obj,limb_branch_dict="all")


 Working on visualization type: mesh

 Working on visualization type: skeleton


In [8]:
pru.exc_high_degree_branching_filter["filter_function"]

<function proofreading_utils.filter_away_high_degree_branching(neuron_obj, return_error_info=False, plot_limb_branch_filter_with_disconnect_effect=False, plot_limb_branch_filter_away=False, plot_final_neuron=False, **kwargs)>

In [9]:
import error_detection as ed
output_info = pru.filter_away_limb_branch_dict_with_function(neuron_obj,
                 limb_branch_dict_function=ed.high_degree_branch_errors_limb_branch_dict,
                return_error_info=True,
                plot_limb_branch_filter_away=True,
                plot_limb_branch_filter_with_disconnect_effect=True,
                 plot_final_neuron=False,
                return_limb_branch_dict_to_cancel=True,)



 ----- Working on coordinate 0: [1139880.  579306. 1048194.]--------
kiss_check = True
winning_downstream = None,error_downstream = [] 
coordinate [1139880.  579306. 1048194.] had error branches []--------


 ----- Working on coordinate 1: [1142791.55987193  642687.69846821 1053519.58719435]--------
kiss_check = True
Changing max_degree_to_resolve = 8 because upstream width was 206.069558989696 
short_thick_endnodes_to_remove = [  9  23  24  25  46  73  80  85  87 104 106 109 112]
winning_downstream = 16,error_downstream = [12 17] 
coordinate [1142791.55987193  642687.69846821 1053519.58719435] had error branches [12 17]--------


 ----- Working on coordinate 2: [1143307.14466577  647958.48340038 1054819.23265028]--------
kiss_check = True
short_thick_endnodes_to_remove = [  9  23  24  25  46  73  80  85  87 104 106 109 112]
winning_downstream = 18,error_downstream = [10 19] 
coordinate [1143307.14466577  647958.48340038 1054819.23265028] had error branches [10 19]--------


 ----- W



Branches Requested to Remove (WITH disconnect effect)

 Working on visualization type: mesh


# Doing the Red blue suggestions

In [ ]:
#apply_proofreading_filters_to_neuron is where do the disconnect

In [12]:
limb_branch_dict_to_cancel = output_info[-1]

In [13]:
red_blue_suggestions = pru.limb_branch_dict_to_cancel_to_red_blue_groups(neuron_obj=neuron_obj,
                                                limb_branch_dict_to_cancel=limb_branch_dict_to_cancel,
                                                            plot_all_blue_red_groups = True,
                                                            plot_final_blue_red_points = True,
                                                            red_blue_points_method = "closest_mesh_face",
                                                             verbose = True)

-- Working on L1 --
Error Branches = [ 12  17  10  19  33  56  76  77  79  84  89  95  98  99  93 105]

**conn_comp_errors = [[33], [98], [99], [105], [10], [12], [76], [77], [79], [17], [19], [84], [56], [89], [93], [95]]
Working on connected component: [33]
Branch 33 had a valid upstream node 57 with common endpoint [[1167531.54936081  749545.32987519 1092171.4209236 ]]
valid_upstream_branches = [57]
error_border_branches = [33]
error_border_coordinates = [array([[1167531.54936081,  749545.32987519, 1092171.4209236 ]])]

Valid Branch 57 had a valid downstream node 55 with common endpoint [[1167531.54936081  749545.32987519 1092171.4209236 ]]

After Adding Downstream Valid Nodes
error_border_branches = [33, -1]
valid_upstream_branches = [57, 55]
error_border_coordinates = [array([[1167531.54936081,  749545.32987519, 1092171.4209236 ]]), array([[1167531.54936081,  749545.32987519, 1092171.4209236 ]])]

Error Branch 33 had an error downstream node 32 with common endpoint [[1151991.   75

Working on connected component: [98]
Branch 98 had a valid upstream node 88 with common endpoint [[1226095.88433037  752675.53239568 1054634.55775811]]
valid_upstream_branches = [88]
error_border_branches = [98]
error_border_coordinates = [array([[1226095.88433037,  752675.53239568, 1054634.55775811]])]

Valid Branch 88 had a valid downstream node 100 with common endpoint [[1226095.88433037  752675.53239568 1054634.55775811]]

After Adding Downstream Valid Nodes
error_border_branches = [98, -1]
valid_upstream_branches = [88, 100]
error_border_coordinates = [array([[1226095.88433037,  752675.53239568, 1054634.55775811]]), array([[1226095.88433037,  752675.53239568, 1054634.55775811]])]


After Adding Downstream Error Nodes
error_border_branches = [98, -1]
valid_upstream_branches = [88, 100]
error_border_coordinates = [array([[1226095.88433037,  752675.53239568, 1054634.55775811]]), array([[1226095.88433037,  752675.53239568, 1054634.55775811]])]

For Pair 0: coordinate [[1226095.8843303

Working on connected component: [99]
Branch 99 had a valid upstream node 88 with common endpoint [[1226095.88433037  752675.53239568 1054634.55775811]]
valid_upstream_branches = [88]
error_border_branches = [99]
error_border_coordinates = [array([[1226095.88433037,  752675.53239568, 1054634.55775811]])]

Valid Branch 88 had a valid downstream node 100 with common endpoint [[1226095.88433037  752675.53239568 1054634.55775811]]

After Adding Downstream Valid Nodes
error_border_branches = [99, -1]
valid_upstream_branches = [88, 100]
error_border_coordinates = [array([[1226095.88433037,  752675.53239568, 1054634.55775811]]), array([[1226095.88433037,  752675.53239568, 1054634.55775811]])]

Error Branch 99 had an error downstream node 97 with common endpoint [[1226371.   750978.9 1054649. ]]
Error Branch 99 had an error downstream node 102 with common endpoint [[1226371.   750978.9 1054649. ]]

After Adding Downstream Error Nodes
error_border_branches = [99, -1, 97, 102]
valid_upstream_bran

Working on connected component: [105]
Branch 105 had a valid upstream node 107 with common endpoint [[1227815.02184284  743819.62598076 1050354.17763248]]
valid_upstream_branches = [107]
error_border_branches = [105]
error_border_coordinates = [array([[1227815.02184284,  743819.62598076, 1050354.17763248]])]

Valid Branch 107 had a valid downstream node 90 with common endpoint [[1227815.02184284  743819.62598076 1050354.17763248]]

After Adding Downstream Valid Nodes
error_border_branches = [105, -1]
valid_upstream_branches = [107, 90]
error_border_coordinates = [array([[1227815.02184284,  743819.62598076, 1050354.17763248]]), array([[1227815.02184284,  743819.62598076, 1050354.17763248]])]

Error Branch 105 had an error downstream node 94 with common endpoint [[1226774.  744542. 1051396.]]
Error Branch 105 had an error downstream node 101 with common endpoint [[1226774.  744542. 1051396.]]

After Adding Downstream Error Nodes
error_border_branches = [105, -1, 94, 101]
valid_upstream_b

Working on connected component: [10]
Branch 10 had a valid upstream node 16 with common endpoint [[1143307.14466577  647958.48340038 1054819.23265028]]
valid_upstream_branches = [16]
error_border_branches = [10]
error_border_coordinates = [array([[1143307.14466577,  647958.48340038, 1054819.23265028]])]

Valid Branch 16 had a valid downstream node 18 with common endpoint [[1143307.14466577  647958.48340038 1054819.23265028]]

After Adding Downstream Valid Nodes
error_border_branches = [10, -1]
valid_upstream_branches = [16, 18]
error_border_coordinates = [array([[1143307.14466577,  647958.48340038, 1054819.23265028]]), array([[1143307.14466577,  647958.48340038, 1054819.23265028]])]

Error Branch 10 had an error downstream node 0 with common endpoint [[1139615.   648271.8 1060584. ]]
Error Branch 10 had an error downstream node 4 with common endpoint [[1139615.   648271.8 1060584. ]]

After Adding Downstream Error Nodes
error_border_branches = [10, -1, 0, 4]
valid_upstream_branches = [

...

Working on connected component: [56]
Branch 56 had a valid upstream node 57 with common endpoint [[1167531.54936081  749545.32987519 1092171.4209236 ]]
valid_upstream_branches = [57]
error_border_branches = [56]
error_border_coordinates = [array([[1167531.54936081,  749545.32987519, 1092171.4209236 ]])]

Valid Branch 57 had a valid downstream node 55 with common endpoint [[1167531.54936081  749545.32987519 1092171.4209236 ]]

After Adding Downstream Valid Nodes
error_border_branches = [56, -1]
valid_upstream_branches = [57, 55]
error_border_coordinates = [array([[1167531.54936081,  749545.32987519, 1092171.4209236 ]]), array([[1167531.54936081,  749545.32987519, 1092171.4209236 ]])]


After Adding Downstream Error Nodes
error_border_branches = [56, -1]
valid_upstream_branches = [57, 55]
error_border_coordinates = [array([[1167531.54936081,  749545.32987519, 1092171.4209236 ]]), array([[1167531.54936081,  749545.32987519, 1092171.4209236 ]])]

For Pair 0: coordinate [[1167531.54936081  

Working on connected component: [89]
Branch 89 had a valid upstream node 90 with common endpoint [[1214258.0878685   719470.78608745 1043268.48229103]]
valid_upstream_branches = [90]
error_border_branches = [89]
error_border_coordinates = [array([[1214258.0878685 ,  719470.78608745, 1043268.48229103]])]

Valid Branch 90 had a valid downstream node 86 with common endpoint [[1214258.0878685   719470.78608745 1043268.48229103]]

After Adding Downstream Valid Nodes
error_border_branches = [89, -1]
valid_upstream_branches = [90, 86]
error_border_coordinates = [array([[1214258.0878685 ,  719470.78608745, 1043268.48229103]]), array([[1214258.0878685 ,  719470.78608745, 1043268.48229103]])]


After Adding Downstream Error Nodes
error_border_branches = [89, -1]
valid_upstream_branches = [90, 86]
error_border_coordinates = [array([[1214258.0878685 ,  719470.78608745, 1043268.48229103]]), array([[1214258.0878685 ,  719470.78608745, 1043268.48229103]])]

For Pair 0: coordinate [[1214258.0878685   

Working on connected component: [93]
Branch 93 had a valid upstream node 107 with common endpoint [[1227815.02184284  743819.62598076 1050354.17763248]]
valid_upstream_branches = [107]
error_border_branches = [93]
error_border_coordinates = [array([[1227815.02184284,  743819.62598076, 1050354.17763248]])]

Valid Branch 107 had a valid downstream node 90 with common endpoint [[1227815.02184284  743819.62598076 1050354.17763248]]

After Adding Downstream Valid Nodes
error_border_branches = [93, -1]
valid_upstream_branches = [107, 90]
error_border_coordinates = [array([[1227815.02184284,  743819.62598076, 1050354.17763248]]), array([[1227815.02184284,  743819.62598076, 1050354.17763248]])]


After Adding Downstream Error Nodes
error_border_branches = [93, -1]
valid_upstream_branches = [107, 90]
error_border_coordinates = [array([[1227815.02184284,  743819.62598076, 1050354.17763248]]), array([[1227815.02184284,  743819.62598076, 1050354.17763248]])]

For Pair 0: coordinate [[1227815.02184

Working on connected component: [95]
Branch 95 had a valid upstream node 88 with common endpoint [[1226095.88433037  752675.53239568 1054634.55775811]]
valid_upstream_branches = [88]
error_border_branches = [95]
error_border_coordinates = [array([[1226095.88433037,  752675.53239568, 1054634.55775811]])]

Valid Branch 88 had a valid downstream node 100 with common endpoint [[1226095.88433037  752675.53239568 1054634.55775811]]

After Adding Downstream Valid Nodes
error_border_branches = [95, -1]
valid_upstream_branches = [88, 100]
error_border_coordinates = [array([[1226095.88433037,  752675.53239568, 1054634.55775811]]), array([[1226095.88433037,  752675.53239568, 1054634.55775811]])]

Error Branch 95 had an error downstream node 78 with common endpoint [[1225306.   753814.1 1055402. ]]
Error Branch 95 had an error downstream node 96 with common endpoint [[1225306.   753814.1 1055402. ]]

After Adding Downstream Error Nodes
error_border_branches = [95, -1, 78, 96]
valid_upstream_branch


 Working on visualization type: mesh

 Working on visualization type: skeleton
